In [1]:
import sys, os
sys.path.append(os.getcwd())
from LIMEaid.model import loadcollegedataset as gcd
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'LIMEaid.LIMEaid'

In [ ]:
gcd.loadcollegedataset()